# Data Wrangling Competition

## Step 1) Make features ##
- `INPUT`: 클릭스트림 데이터(`train_clickstreams.tab`, `test_clickstreams.tab`)
- `OUTPUT`: 위 데이터로부터 파생변수(feature)를 만든 후 모형 학습용(`X_train.csv`)과 평가용(`X_test.csv`) 데이터 생성

In [1]:
%run import_modules.py
%matplotlib inline

*학습용과 제출용 클릭스트림 데이터를 읽고 하나로 합친다.*

In [2]:
train = pd.read_csv('train_clickstreams.tab', sep='\t', encoding='cp949', dtype={'TIME_ID': str})
test = pd.read_csv('test_clickstreams.tab', sep='\t', encoding='cp949', dtype={'TIME_ID': str})
tr = pd.concat([train, test]); tr

,CUS_ID,TIME_ID,SITE,SITE_CNT,ST_TIME,SITE_NM,BACT_NM,MACT_NM,ACT_NM
0,1,2012070905,search.naver.com,3.0,794.0,네이버 검색,인터넷/컴퓨터,검색,포털검색
1,1,2012072507,plus.google.com,1.0,1.0,구글 Plus,커뮤니티,블로그/SNS,SNS
2,1,2012081116,joongang.joinsmsn.com,2.0,5.0,중앙일보,뉴스/미디어,일간지,종합일간지
3,1,2012090304,news.naver.com,5.0,504.0,네이버 뉴스,뉴스/미디어,인터넷신문,포털뉴스
4,1,2012090506,news.nate.com,1.0,0.0,네이트 뉴스,뉴스/미디어,인터넷신문,포털뉴스
...,...,...,...,...,...,...,...,...,...
8522119,5000,2012101804,www.naver.com,2.0,11.0,네이버,인터넷/컴퓨터,포털,종합포털
8522120,5000,2012110215,www.amazon.com,5.0,344.0,Amazon,쇼핑,종합쇼핑,종합쇼핑몰
8522121,5000,2012111614,www.kyobobook.co.kr,16.0,334.0,교보문고,쇼핑,도서/음반/악기 쇼핑몰,종합도서쇼핑몰
8522122,5000,2013010221,www.naver.com,6.0,0.0,네이버,인터넷/컴퓨터,포털,종합포털


In [3]:
# 날짜시간이 문자로 된 컬럼을 datatime 형식으로 변환
tr.TIME_ID = pd.to_datetime(tr.TIME_ID, format='%Y%m%d%H')

*파생변수를 저장할 빈 리스트를 만든다.*

In [4]:
features = []

<br><font color='green'>
### Numeric Features

**[파생변수 1-2]** 총페이지뷰, 평균페이지뷰

In [5]:
f = tr.groupby('CUS_ID')['SITE_CNT'].agg([
    ('총페이지뷰',np.sum), 
    ('평균페이지뷰', lambda x: np.round(np.mean(x))),
]).reset_index()
features.append(f); f

,CUS_ID,총페이지뷰,평균페이지뷰
0,1,13656.0,4.0
1,2,25498.0,5.0
2,3,10413.0,5.0
3,4,12831.0,7.0
4,5,13267.0,3.0
...,...,...,...
4995,4996,17627.0,8.0
4996,4997,8366.0,7.0
4997,4998,2253.0,5.0
4998,4999,7904.0,5.0


**[파생변수 3-6]** (서로다른) 접속사이트: SITE_NM, BACT_NM(대분류), MACT_NM(중분류), ACT_NM(소분류) 각각

In [6]:
f = tr.groupby('CUS_ID').agg({
    'SITE_NM': [('접속종류_사이트', lambda x: x.nunique())],
    'ACT_NM':  [('접속종류_소분류', lambda x: x.nunique())],
    'MACT_NM': [('접속종류_중분류', lambda x: x.nunique())],
    'BACT_NM': [('접속종류_대분류', lambda x: x.nunique())],
})
f.columns = f.columns.droplevel()
f=f.reset_index()
features.append(f); f

,CUS_ID,접속종류_사이트,접속종류_소분류,접속종류_중분류,접속종류_대분류
0,1,201,91,52,16
1,2,573,182,90,20
2,3,270,96,53,17
3,4,166,85,50,17
4,5,328,139,73,20
...,...,...,...,...,...
4995,4996,208,107,59,17
4996,4997,194,104,64,19
4997,4998,67,45,34,14
4998,4999,169,91,49,18


**[파생변수 7-13]** 이용일수, 이용주기, 주말이용비율, 계절별 이용비율(Spring(3-5)/Summer(6-8)/Fall(9-11)/Winter(12-2))

In [7]:
f = tr.groupby('CUS_ID')['TIME_ID'].agg([
    ('이용일수',lambda x: x.dt.date.nunique()),
    ('이용주기', lambda x: int((x.max() - x.min()).days / x.dt.date.nunique())),
    ('이용률_주말', lambda x: np.mean(x.dt.dayofweek>4)),
    ('이용률_봄', lambda x: np.mean(x.dt.month.isin([3,4,5]))),
    ('이용률_여름', lambda x: np.mean(x.dt.month.isin([6,7,8]))),
    ('이용률_가을', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
    ('이용률_겨울', lambda x: np.mean(x.dt.month.isin([1,2,12])))
]).reset_index()
features.append(f); f

,CUS_ID,이용일수,이용주기,이용률_주말,이용률_봄,이용률_여름,이용률_가을,이용률_겨울
0,1,205,1,0.242812,0.054365,0.685050,0.203084,0.057501
1,2,105,3,0.011271,0.366778,0.276608,0.176275,0.180340
2,3,105,2,0.071173,0.000000,0.326768,0.504029,0.169203
3,4,107,1,0.107270,0.000000,0.608541,0.294865,0.096594
4,5,151,2,0.110738,0.169463,0.292186,0.263663,0.274688
...,...,...,...,...,...,...,...,...
4995,4996,132,2,0.010652,0.202841,0.181980,0.335109,0.280071
4996,4997,112,3,0.291469,0.168246,0.136651,0.249605,0.445498
4997,4998,113,1,0.434322,0.000000,0.290254,0.542373,0.167373
4998,4999,166,1,0.212892,0.335305,0.077469,0.424009,0.163217


**[파생변수 14-35]** 사이트 대분류별(BACT_NM) 총접속시간

In [8]:
f = pd.pivot_table(tr, index='CUS_ID', columns='BACT_NM', values='ST_TIME', aggfunc=np.sum, fill_value=0).reset_index()
features.append(f); f

BACT_NM,CUS_ID,건강/의학,게임,교육/학원,금융/부동산,뉴스/미디어,문학/예술,비즈니스/경제,사회/문화/종교,생활/가정/취미,...,엔터테인먼트,여행,온라인교육,유통/판매/운송,인터넷/컴퓨터,정보통신/IT,정치/행정,제조,커뮤니티,학문
0,1,0,244,109,213,100098,57,2136,12,0,...,73388,453,5335,0,94403,0,14,0,78843,0
1,2,229,439,1070,9259,23034,60,74483,60,2934,...,12663,2434,555,1507,293318,4963,665,966,74538,0
2,3,2670,0,121,312,7185,0,2734,60,1492,...,2867,2748,0,62,282450,427,846,217,11175,0
3,4,2,1,0,10310,71608,0,2012,0,0,...,1072,13,26,146,102907,2041,3117,522,65843,0
4,5,192,1021,471,23216,100285,0,41,1,1199,...,1496,1578,3567,143,287104,538,11491,47,38769,188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,1,0,0,21336,24969,0,20787,0,1809,...,581,241,795,601,315302,1618,3038,384,8260,0
4996,4997,65,3480,0,1497,10876,0,2408,22,82,...,1690,9986,8,105,44814,132,1553,910,122403,0
4997,4998,156,44211,8,620,315,0,3022,0,0,...,264,156,0,0,23704,230,0,259,4131,0
4998,4999,49,93,48800,0,3214,0,2052,727,207,...,128,275,23002,47,50303,3944,254,793,12455,0


**[파생변수 36-39]** 최근 1개월, 3개월 이용시간 및 이용건수

In [9]:
# pd.to_datetime(), pd.offsets.MonthBegin(3)
for m in [1,3]:
    start = tr.TIME_ID.max() - pd.offsets.MonthBegin(m)
    f = tr.query('TIME_ID >= @start').groupby('CUS_ID')['ST_TIME'].agg([
        (f'최근{m}개월_이용시간', np.sum), 
        (f'최근{m}개월_이용건수', np.size)
    ]).reset_index()
    display(f)
    features.append(f)

,CUS_ID,최근1개월_이용시간,최근1개월_이용건수
0,1,26097.0,387.0
1,2,119868.0,739.0
2,6,952.0,6.0
3,7,60559.0,105.0
4,8,3487.0,68.0
...,...,...,...
3544,4993,38545.0,201.0
3545,4995,65523.0,388.0
3546,4996,11083.0,52.0
3547,4997,1568.0,17.0


,CUS_ID,최근3개월_이용시간,최근3개월_이용건수
0,1,38113.0,545.0
1,2,323884.0,2075.0
2,5,24523.0,286.0
3,6,12860.0,76.0
4,7,233213.0,598.0
...,...,...,...
4166,4995,127746.0,1202.0
4167,4996,47394.0,189.0
4168,4997,12261.0,124.0
4169,4999,49589.0,476.0


<br><font color='green'>
### Categorical Features

**[파생변수 40]** 주이용사이트 (소분류 기준)

In [10]:
f = tr.groupby('CUS_ID')['ACT_NM'].agg([('주이용사이트', lambda x: x.value_counts().index[0])]).reset_index()
features.append(f); f

,CUS_ID,주이용사이트
0,1,종합포털
1,2,포털검색
2,3,종합포털
3,4,종합포털
4,5,종합포털
...,...,...
4995,4996,종합포털
4996,4997,종합포털
4997,4998,게임포털
4998,4999,종합포털


<br><font color='green'>
### 기타

Kaggle Competition에서 자주 사용되는 feature 생성방법(아래 pdf 참조)을 참고하기 바람.
- https://drive.google.com/open?id=1HDZc1mDvtmpjg9YPpUN0koHeAiqTAHRw

<br><br>
*아래 코드를 수행하면 생성한 모든 파생변수가 병합되고 학습용과 평가용으로 나뉘어진다.*

In [11]:
X_train = DataFrame({'CUS_ID': np.sort(train['CUS_ID'].unique())})
for f in features :
    X_train = pd.merge(X_train, f, how='left')
display(X_train)

X_test = DataFrame({'CUS_ID': np.sort(test['CUS_ID'].unique())})
for f in features :
    X_test = pd.merge(X_test, f, how='left')
display(X_test)

,CUS_ID,총페이지뷰,평균페이지뷰,접속종류_사이트,접속종류_소분류,접속종류_중분류,접속종류_대분류,이용일수,이용주기,이용률_주말,...,정보통신/IT,정치/행정,제조,커뮤니티,학문,최근1개월_이용시간,최근1개월_이용건수,최근3개월_이용시간,최근3개월_이용건수,주이용사이트
0,1,13656.0,4.0,201,91,52,16,205,1,0.242812,...,0,14,0,78843,0,26097.0,387.0,38113.0,545.0,종합포털
1,2,25498.0,5.0,573,182,90,20,105,3,0.011271,...,4963,665,966,74538,0,119868.0,739.0,323884.0,2075.0,포털검색
2,3,10413.0,5.0,270,96,53,17,105,2,0.071173,...,427,846,217,11175,0,NaN,NaN,NaN,NaN,종합포털
3,4,12831.0,7.0,166,85,50,17,107,1,0.107270,...,2041,3117,522,65843,0,NaN,NaN,NaN,NaN,종합포털
4,5,13267.0,3.0,328,139,73,20,151,2,0.110738,...,538,11491,47,38769,188,NaN,NaN,24523.0,286.0,종합포털
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2496,10349.0,5.0,170,85,56,20,113,1,0.057077,...,415,4572,152,43777,12,40955.0,300.0,136617.0,917.0,종합일간지
2496,2497,8419.0,4.0,270,118,64,18,138,1,0.327929,...,295,667,3,37325,0,19880.0,230.0,75671.0,825.0,종합포털
2497,2498,24179.0,5.0,431,145,70,19,171,2,0.195558,...,299,119,316,68467,29,15763.0,166.0,76756.0,690.0,종합포털
2498,2499,12541.0,5.0,227,124,73,19,114,3,0.279829,...,379,4581,38,38276,24,52888.0,313.0,160013.0,1074.0,종합포털


,CUS_ID,총페이지뷰,평균페이지뷰,접속종류_사이트,접속종류_소분류,접속종류_중분류,접속종류_대분류,이용일수,이용주기,이용률_주말,...,정보통신/IT,정치/행정,제조,커뮤니티,학문,최근1개월_이용시간,최근1개월_이용건수,최근3개월_이용시간,최근3개월_이용건수,주이용사이트
0,2501,16733.0,6.0,230,113,61,17,122,2,0.156832,...,4424,2977,472,74502,0,14701.0,115.0,35890.0,306.0,종합포털
1,2502,20391.0,5.0,344,163,76,19,136,2,0.004044,...,2152,4597,2117,9299,0,NaN,NaN,84384.0,527.0,종합포털
2,2503,12002.0,8.0,137,76,48,15,139,1,0.259185,...,250,0,2026,96270,0,NaN,NaN,NaN,NaN,종합포털
3,2504,20121.0,5.0,245,105,59,18,169,1,0.288382,...,1413,10995,176,115135,0,NaN,NaN,NaN,NaN,종합포털
4,2505,3108.0,4.0,110,64,40,13,99,3,0.256776,...,0,824,24,9958,0,40.0,1.0,10070.0,42.0,종합포털
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,4996,17627.0,8.0,208,107,59,17,132,2,0.010652,...,1618,3038,384,8260,0,11083.0,52.0,47394.0,189.0,종합포털
2496,4997,8366.0,7.0,194,104,64,19,112,3,0.291469,...,132,1553,910,122403,0,1568.0,17.0,12261.0,124.0,종합포털
2497,4998,2253.0,5.0,67,45,34,14,113,1,0.434322,...,230,0,259,4131,0,NaN,NaN,NaN,NaN,게임포털
2498,4999,7904.0,5.0,169,91,49,18,166,1,0.212892,...,3944,254,793,12455,0,19707.0,130.0,49589.0,476.0,종합포털


In [12]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 0 to 2499
Data columns (total 41 columns):
CUS_ID        2500 non-null int64
총페이지뷰         2500 non-null float64
평균페이지뷰        2500 non-null float64
접속종류_사이트      2500 non-null int64
접속종류_소분류      2500 non-null int64
접속종류_중분류      2500 non-null int64
접속종류_대분류      2500 non-null int64
이용일수          2500 non-null int64
이용주기          2500 non-null int64
이용률_주말        2500 non-null float64
이용률_봄         2500 non-null float64
이용률_여름        2500 non-null float64
이용률_가을        2500 non-null float64
이용률_겨울        2500 non-null float64
건강/의학         2500 non-null int64
게임            2500 non-null int64
교육/학원         2500 non-null int64
금융/부동산        2500 non-null int64
뉴스/미디어        2500 non-null int64
문학/예술         2500 non-null int64
비즈니스/경제       2500 non-null int64
사회/문화/종교      2500 non-null int64
생활/가정/취미      2500 non-null int64
서비스           2500 non-null int64
쇼핑            2500 non-null int64
스포츠/레저        2500 non-null i

*아래 코드를 수행하면 학습용 데이터와 평가용 데이터가 저장된다.*
<font color='red'>

In [13]:
X_train.to_csv('X_train.csv', index=False, encoding='cp949')
X_test.to_csv('X_test.csv', index=False, encoding='cp949')

# End